In [1]:
!pip install PyMuPDF Pillow pytesseract requests sparknlp
!sudo apt update
!sudo apt install -y tesseract-ocr libtesseract-dev

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.0/713.0 kB 35.0 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [79.8 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,798 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:11 https://ppa.launchpadcontent.

In [2]:
!pip install spark-nlp-display

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.9 MB/s eta 0:00:00


In [3]:
from sparknlp_display import NerVisualizer


In [4]:
# !wget https://setup.johnsnowlabs.com/colab.sh -O - | bash -s -- -p 3.5.0

In [5]:
import sparknlp
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, explode
from pyspark.sql.types import StringType
from sparknlp.base import DocumentAssembler, Pipeline
from sparknlp.annotator import SentenceDetectorDLModel, Tokenizer, WordEmbeddingsModel, NerDLModel, NerConverter

import fitz  # PyMuPDF
from PIL import Image
import pytesseract
import requests
import io
import re

pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

# Start Spark NLP session
spark = sparknlp.start()
print(f"Spark NLP version: {sparknlp.version()}")
print(f"Spark version: {spark.version}")



Spark NLP version: 6.0.3
Spark version: 3.5.1


In [6]:
def ocr_pdf_to_text(pdf_path_or_url):
    all_text = ""
    doc = None

    if pdf_path_or_url.startswith("http"):
        response = requests.get(pdf_path_or_url)
        response.raise_for_status()
        doc = fitz.open(stream=response.content, filetype="pdf")
    else:
        doc = fitz.open(pdf_path_or_url)

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        pix = page.get_pixmap(matrix=fitz.Matrix(2, 2))
        img = Image.open(io.BytesIO(pix.tobytes()))
        page_text = pytesseract.image_to_string(img, lang='eng')
        all_text += page_text + "\n"
    doc.close()
    return all_text


In [7]:
from google.colab import files

uploaded = files.upload()

pdf_file = list(uploaded.keys())[0]

print(f"Uploaded file: {pdf_file}")

extracted_text = ocr_pdf_to_text(pdf_file)
print("Extracted text preview (first 500 chars):\n", extracted_text[:500])




Saving MT_OCR_00.pdf to MT_OCR_00.pdf
Uploaded file: MT_OCR_00.pdf
Extracted text preview (first 500 chars):
 Sample Type / Medical Specialty:
Hematology - Oncology

Sample Name:
Discharge Summary - Mesothelioma — 1

Record date : 2093-01-13 , David Hale, M.D .,

Name : John Hendrickson # 7194334

Description:

Mesothelioma, pleural effusion, atrial fibrillation, anemia, ascites, esophageal reflux, and
history of deep venous thrombosis.

(Medical Transcription Sample Report)

PRINCIPAL DIAGNOSIS:
Mesothelioma.

SECONDARY DIAGNOSES:
Pleural effusion, atrial fibrillation, anemia, ascites, esophageal reflu


In [8]:
df = spark.createDataFrame([[extracted_text]]).toDF("text")


In [9]:
documentAssembler = DocumentAssembler().setInputCol("text").setOutputCol("document")
sentenceDetector = SentenceDetectorDLModel.pretrained("sentence_detector_dl", "en")\
    .setInputCols(["document"]).setOutputCol("sentence")
tokenizer = Tokenizer().setInputCols(["sentence"]).setOutputCol("token")
word_embeddings = WordEmbeddingsModel.pretrained("glove_100d").setInputCols(["document", "token"]).setOutputCol("embeddings")
ner_model = NerDLModel.pretrained("ner_dl", "en").setInputCols(["sentence", "token", "embeddings"]).setOutputCol("ner")
ner_converter = NerConverter().setInputCols(["sentence", "token", "ner"]).setOutputCol("entities")

pipeline = Pipeline(stages=[
    documentAssembler,
    sentenceDetector,
    tokenizer,
    word_embeddings,
    ner_model,
    ner_converter
])

model = pipeline.fit(df)
result_df = model.transform(df)


sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]
ner_dl download started this may take some time.
Approximate size to download 13.6 MB
[OK!]


In [19]:
from sparknlp.base import LightPipeline

light_pipeline = LightPipeline(pipeline.model)


In [12]:

# print(result[0]['entities'])


In [20]:
result_df.select(
    explode(col("entities")).alias("entity")
).select(
    col("entity.result").alias("entity_text"),
    col("entity.metadata.entity").alias("entity_type")
).show(truncate=False)



+-----------------------------------+-----------+
|entity_text                        |entity_type|
+-----------------------------------+-----------+
|/ Medical Specialty:\nHematology   |ORG        |
|Oncology                           |ORG        |
|Sample                             |ORG        |
|Mesothelioma                       |PER        |
|David Hale                         |PER        |
|M.D                                |PER        |
|John Hendrickson                   |PER        |
|Description                        |MISC       |
|Mesothelioma                       |PER        |
|Medical Transcription Sample Report|ORG        |
|Mesothelioma                       |PER        |
|HISTORY AND PHYSICAL               |MISC       |
|Vietnam                            |LOC        |
|Chest                              |PER        |
|PAST MEDICAL HISTORY               |ORG        |
|Pericardectomy                     |PER        |
|Pericarditis                       |PER        |


In [21]:

from pyspark.sql.functions import explode, col

# Extract entities and entity types as a Pandas DataFrame for visualization
pandas_df = result_df.select(
    explode(col("entities")).alias("entity")
).select(
    col("entity.result").alias("entity_text"),
    col("entity.metadata.entity").alias("entity_type")
).toPandas()

display(pandas_df)


,entity_text,entity_type
0,/ Medical Specialty:\nHematology,ORG
1,Oncology,ORG
2,Sample,ORG
3,Mesothelioma,PER
4,David Hale,PER
5,M.D,PER
6,John Hendrickson,PER
7,Description,MISC
8,Mesothelioma,PER
9,Medical Transcription Sample Report,ORG


In [22]:
from sparknlp_display import NerVisualizer

# Annotate using LightPipeline
pipeline_result = light_pipeline.fullAnnotate(extracted_text)

# Inspect the annotation structure
for key, value in pipeline_result[0].items():
    print(f"\nKEY: {key}")
    if isinstance(value, list):
        for item in value[:2]:  # Just the first 2 items for brevity
            print(item)
    else:
        print(value)

ner_vis = NerVisualizer()
ner_vis.display(pipeline_result[0], label_col='entities', document_col='document')




KEY: entities
Annotation(chunk, 14, 30, Medical Specialty, {'entity': 'ORG', 'sentence': '0', 'chunk': '0'}, [])
Annotation(chunk, 89, 100, Mesothelioma, {'entity': 'PER', 'sentence': '0', 'chunk': '1'}, [])

KEY: document
Annotation(document, 0, 5619, Sample Type / Medical Specialty:
Hematology - Oncology

Sample Name:
Discharge Summary - Mesothelioma — 1

Record date : 2093-01-13 , David Hale, M.D .,

Name : John Hendrickson # 7194334

Description:

Mesothelioma, pleural effusion, atrial fibrillation, anemia, ascites, esophageal reflux, and
history of deep venous thrombosis.

(Medical Transcription Sample Report)

PRINCIPAL DIAGNOSIS:
Mesothelioma.

SECONDARY DIAGNOSES:
Pleural effusion, atrial fibrillation, anemia, ascites, esophageal reflux, and history of deep
venous thrombosis.

PROCEDURES

1. On August 24, 2007, decortication of the lung with pleural biopsy and transpleural
fluoroscopy.

2. On August 20, 2007, thoracentesis.

3. On August 31, 2007, Port-A-Cath placement.

HISTO

In [16]:
@udf(StringType())
def mask_entities(text, entities):
    if not entities:
        return text
    text_chars = list(text)
    # Sort entities by start index descending to avoid index shifting issues
    sorted_entities = sorted(entities, key=lambda x: x.begin, reverse=True)
    for e in sorted_entities:
        entity_type = e.metadata.get('entity', 'UNKNOWN').upper()
        start = e.begin
        end = e.end
        if 0 <= start < end <= len(text_chars):
            placeholder = f"<{entity_type}>"
            text_chars[start:end] = list(placeholder)
    return "".join(text_chars)


In [17]:
deid_df = result_df.withColumn("deidentified_text", mask_entities(col("text"), col("entities")))

deid_df.select("deidentified_text").show(truncate=False)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------